In [1]:
import configparser
from pathlib import Path
import sys

import numpy as np
from tensorflow import keras
from keras_applications import correct_pad

import shunt_connector

s_and_e_location = 'mnv3'
#s_and_e_location = 'en-de'
s_and_e_ratio = 0.25

In [2]:
config_path = Path("config", "standard.cfg")
config = configparser.ConfigParser()
config.read(config_path)

connector = shunt_connector.ShuntConnector(config)

connector.create_dataset()
connector.create_original_model()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)

Create dataset
CIFAR10 was loaded successfully!

Create original model
MobileNetV3Small created successfully!
{'conv2d': 15128256.0, 'depthwise_conv2d': 2436096.0, 'total': 17564352.0}


In [3]:

# Firstly, calculate properties like input & output shapes, stride layers, and dilation rates
input_shape = connector.original_model.get_layer(index=connector.shunt_params['locations'][0]).input_shape[1:]
if isinstance(input_shape, list):
    input_shape = input_shape[0][1:]
output_shape = connector.original_model.get_layer(index=connector.shunt_params['locations'][1]).output_shape[1:]
if isinstance(output_shape, list):
    output_shape = output_shape[0][1:]

num_stride_layers = np.round(np.log2(input_shape[1] / output_shape[1]))
dilation_rates = shunt_connector.shunt.find_dilation_rates.find_input_output_dilation_rates(connector.original_model, connector.shunt_params['locations'])

# The predefined s_and_e blocks from MobileNetV3 are used. They are implemented in the keras_application module.

with connector.activate_distribution_scope():
    # shunt model
    input_net = keras.layers.Input(shape=input_shape)
    x = input_net

    x = keras.layers.Conv2D(192,
            kernel_size=(1,1),
            strides=(1,1),
            padding='same',
            use_bias=False,
            activation=None,
            name="shunt_conv_1",
            kernel_initializer="he_normal",
            kernel_regularizer=keras.regularizers.l2(4e-5))(x)
    x = keras.layers.BatchNormalization(epsilon=1e-3,
                        momentum=0.999,
                        name="shunt_batch_norm_1")(x)
    x = keras.layers.ReLU(6., name="shunt_relu_1")(x)

    if num_stride_layers > 0:
        x = keras.layers.DepthwiseConv2D(kernel_size=(3,3),
                            strides=(2,2),
                            padding='same',
                            use_bias=False,
                            activation=None,
                            name="shunt_depth_conv_1",
                            kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l2(4e-5))(x)
    else:
        x = keras.layers.DepthwiseConv2D(kernel_size=(3,3),
                            strides=(1,1),
                            dilation_rate=(dilation_rates[0],dilation_rates[0]),
                            padding='same',
                            use_bias=False,
                            activation=None,
                            name="shunt_depth_conv_1",
                            kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l2(4e-5))(x)
    x = keras.layers.BatchNormalization(epsilon=1e-3,
                        momentum=0.999,
                        name="shunt_batch_norm_2")(x)
    x = keras.layers.ReLU(6., name="shunt_relu_2")(x)

    if s_and_e_location == 'mnv3':
        x = shunt_connector.models.mobile_net_v3._se_block(x, filters=192, se_ratio=s_and_e_ratio, prefix="shunt_1/")

    x = keras.layers.Conv2D(64,
            kernel_size=(1,1),
            strides=(1,1),
            padding='same',
            use_bias=False,
            activation=None,
            name="shunt_conv_2",
            kernel_initializer="he_normal",
            kernel_regularizer=keras.regularizers.l2(4e-5))(x)
    x = keras.layers.BatchNormalization(epsilon=1e-3,
                        momentum=0.999,
                        name="shunt_batch_norm_3")(x)

    if s_and_e_location == 'en_de':
        x = shunt_connector.models.mobile_net_v3._se_block(x, filters=64, se_ratio=s_and_e_ratio, prefix="shunt_1/")

    x = keras.layers.Conv2D(192,
            kernel_size=(1,1),
            strides=(1,1),
            padding='same',
            use_bias=False,
            activation=None,
            name="shunt_conv_3",
            kernel_initializer="he_normal",
            kernel_regularizer=keras.regularizers.l2(4e-5))(x)
    x = keras.layers.BatchNormalization(epsilon=1e-3,
                        momentum=0.999,
                        name="shunt_batch_norm_4")(x)
    x = keras.layers.ReLU(6., name="shunt_relu_3")(x)

    if num_stride_layers > 1:
        x = keras.layers.DepthwiseConv2D(kernel_size=(3,3),
                            strides=(2,2),
                            padding='same',
                            use_bias=False,
                            activation=None,
                            name="shunt_depth_conv_2",
                            kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l2(4e-5))(x)
    else:
        x = keras.layers.DepthwiseConv2D(kernel_size=(3,3),
                            strides=(1,1),
                            dilation_rate=(dilation_rates[1],dilation_rates[1]),
                            padding='same',
                            use_bias=False,
                            activation=None,
                            name="shunt_depth_conv_2",
                            kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l2(4e-5))(x)
    x = keras.layers.BatchNormalization(epsilon=1e-3,
                        momentum=0.999,
                        name="shunt_batch_norm_5")(x)
    x = keras.layers.ReLU(6., name="shunt_relu_4")(x)

    if s_and_e_location == 'mnv3':
        x = shunt_connector.models.mobile_net_v3._se_block(x, filters=192, se_ratio=s_and_e_ratio, prefix="shunt_2/")

    x = keras.layers.Conv2D(output_shape[-1],
            kernel_size=(1,1),
            strides=(1,1),
            padding='same',
            use_bias=False,
            activation=None,
            name="shunt_conv_4",
            kernel_initializer="he_normal",
            kernel_regularizer=keras.regularizers.l2(4e-5))(x)
    x = keras.layers.BatchNormalization(epsilon=1e-3,
                        momentum=0.999,
                        name="shunt_batch_norm_6")(x)

    shunt_model = keras.models.Model(inputs=input_net, outputs=x, name='shunt')

connector.set_shunt_model(shunt_model)
connector.print_summary()

AttributeError: module 'shunt_connector.models' has no attribute 'MobileNet_v3'